<a href="https://colab.research.google.com/github/kentokura/gob/blob/main/%E5%8D%92%E8%AB%96%E6%A6%82%E8%A6%81%E6%8F%90%E5%87%BA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Pythonでゴブレットゴブラーズのゲーム木を作成する

- [ゴブレットゴブラーズ (日本語版)](https://sgrk.blog.fc2.com/blog-entry-3687.html)
- [ソースコード設計スライド](https://docs.google.com/presentation/d/1K8fbRlk24Y_J3M--F-WnYYN0hddL89w3EGN4Zgw-L1g/edit?usp=sharing)

- [卒論概要](https://docs.google.com/document/d/1jEXvVmpUPCaowl3xR1qgyhUFGVHFVzyZjBo9F8OAR7M/edit?usp=sharing)

# 必要な関数群

## stateを定義

set_stateとchoise_stateの２種類がある。  
必要な情報を84bitで表す。  
state = 0b_0_0_0_000000_000000000_000000000000000000000000000000000000000000000000000000_000000000000


| 区分 | is_first | is_choise | is_choise_board | choise_piece_type | choise_piece | board | hand |
| ---- | ---- | ---- | ---- | ---- | ---- | ---- | ---- |
| bit数 | 1bit | 1bit | 1bit | 6bit | 9bit | 54bit | 12bit |
| 例 | 1 | 1 | 1 | 100000 | 100000000 | 010000000 001000000 000110000 000000000 000000000 000000000 | 00 00 00 00 00 00 |
| 説明 | 手番 | モード | 持ち上げている駒は盤か手か | 持ち上げている駒の種類[rl,rm,rs,bl,bm,bs] | 持ち上げている駒のマス[0,1,2,3,4,5,6,7,8] | 盤面に置いてある駒[rl0, rl1, ..., rm0..., bl0...] | 手駒[rlの個数, rmの...] |

説明の補足 :   
- hand  “00”...その駒は持ってない, “01”...一つ持ってる, “11”...２つ持ってる


In [2]:
# 確認
state = 0b_1_0_0_100000_100000000_100000000000000000000000000000000000000000000000000000_100000000000
print(state) # 10進数
print(f'{state:084b}')#bin(プレフィックスなし)
print(type(f'{state:084b}')) #文字列です
print(len(f'{state:084b}')) # bit数の確認
# 1bit取り出す
print(f'{state:084b}'[0])
# 複数のbitを取り出す
print(f'{state:084b}'[1+1+1:1+1+1+6])
# 1bit書き換え
print("変更前 : " + f'{state:084b}')
state = int(f'{state:084b}'[:1] + '1' + f'{state:084b}'[2:], 2)
print("変更後 : " + f'{state:084b}')
# 複数のbitを書き換え
print("変更前 : " + f'{state:084b}')
state = int(f'{state:084b}'[:2] + '1' + f'{state:084b}'[3:72] + '111111000000', 2)
print("変更後 : " + f'{state:084b}')

# バラす
is_first = f'{state:084b}'[0]
is_choise = f'{state:084b}'[1]
is_choise_board = f'{state:084b}'[2]
choise_piece_type = f'{state:084b}'[3:3+6]
choise_piece = f'{state:084b}'[9:9+9]
board = f'{state:084b}'[18:18+54]
hand = f'{state:084b}'[72:72+12]
print("バラす  : " + is_first + is_choise + is_choise_board + choise_piece_type + choise_piece + board + hand)

10899258735951288572315648
100100000100000000100000000000000000000000000000000000000000000000000000100000000000
<class 'str'>
84
1
100000
変更前 : 100100000100000000100000000000000000000000000000000000000000000000000000100000000000
変更後 : 110100000100000000100000000000000000000000000000000000000000000000000000100000000000
変更前 : 110100000100000000100000000000000000000000000000000000000000000000000000100000000000
変更後 : 111100000100000000100000000000000000000000000000000000000000000000000000111111000000
バラす  : 111100000100000000100000000000000000000000000000000000000000000000000000111111000000


## actionを定義

set_action, choise_actionともに、行動する駒を表す。

必要な情報を16bitで表す。  
action = 0b_0_000000_000000000


| 区分 | is_choise_board | choise_piece_type | choise_place |
| ---- | ---- | ---- | ---- | 
| bit数 | 1bit | 6bit | 9bit | 
| 例 | 1 | 100000 | 100000000 | 
| 説明 | 行動する駒は盤か手か | 行動する駒の種類[rl,rm,rs,bl,bm,bs] | 行動する駒のマス[0,1,2,3,4,5,6,7,8] | 


In [3]:
# 確認
action = 0b_1_100000_100000000
print(action) # 10進数
print(f'{action:016b}')#bin(プレフィックスなし)
print(type(f'{action:016b}')) #文字列です
print(len(f'{action:016b}')) # bit数の確認
# 1bit取り出す
print(f'{action:016b}'[0])
# 複数のbitを取り出す
print(f'{action:016b}'[1:1+6])
# 書き換え
print("変更前 : " + f'{action:016b}')
action = int(f'{action:016b}'[:1] + '000001' + '000000010', 2)
print("変更後 : " + f'{action:016b}')


# バラす
is_choise_board = f'{action:016b}'[0]
choise_piece_type = f'{action:016b}'[1:1+6]
choise_piece = f'{action:016b}'[7:7+9]
print("バラす  : " + is_choise_board + choise_piece_type + choise_piece)

49408
1100000100000000
<class 'str'>
16
1
100000
変更前 : 1100000100000000
変更後 : 1000001000000010
バラす  : 1000001000000010


## stateからその合法手一覧であるactionsを作成する関数

合法手をすべて列挙する... actions = [action1, action2, …]  
leagal_actions(state) -> actions: を作成
```
def leagal_actions(state) -> actions:
  actions = []
  もし stateがchoise_stateなら、
    actions = leagal_actions_from_choise_state(choise_state)
  そうでなく stateがset_stateなら、
    actions = leagal_actions_from_set_state(set_state)
  return actions
```
stateごとに内部関数を作成。  
- choise_stateから合法手を作成 : leagal_actions_from_choise_state(choise_state)
- set_stateから合法手を作成 : leagal_actions_from_set_state(set_state)

### choise_stateから合法手を作成

  
leagal_actions_from_choise_state(choise_state) -> set_actions:  
setする駒と場所一覧... set_actions を作成する  
setできる駒と場所とは....
- handは行動できない。あくまで選んだ駒を盤面に配置するという作業だから。
- choiseしている駒と同じ駒しか行動できない。例: rlを選んだならrlしか行動できない。
- choiseした駒以上の大きさの駒があるマスには行動できない。
- choiseした駒が盤面なら、
    - choiseした駒があったマスには行動できない。

In [4]:
def leagal_actions_from_choise_state(choise_state):
  is_choise_board = f'{choise_state:084b}'[2]
  choise_piece_type = f'{choise_state:084b}'[3:3+6]
  choise_piece = f'{choise_state:084b}'[9:9+9]
  board = f'{choise_state:084b}'[18:18+54]

  # set_actions_bitを作成
  set_actions_bit = f'{0:054b}' # 54bit。すべての駒と位置が行動不能で初期化
  # choiseしている駒と同じ駒は行動できる。
  cpy_f = choise_piece_type.find('1')
  set_actions_bit = set_actions_bit[:9*cpy_f] + '111111111' + set_actions_bit[9*(cpy_f+1):]
  # choiseしている駒がboardの駒なら、そのマスに行動出来ない
  if is_choise_board == '1': 
    cp_f = choise_piece.find('1')
    set_actions_bit = set_actions_bit[:9*cpy_f+cp_f] + '0' + set_actions_bit[9*cpy_f+cp_f+1:]
  # choiseした駒以上の大きさの駒があるマスには行動できない。
  for mass in range(9):
    if board[mass] == '1' or board[mass+27] == "1": # L駒があるか
      set_actions_bit = set_actions_bit[:9*cpy_f+mass] + '0' + set_actions_bit[9*cpy_f+mass+1:]
    if cpy_f in [1, 2, 4, 5]: #選択してあるこまがM, Sなら
      if board[mass+9] == '1' or board[mass+27+9] == "1": # M駒があるか
        set_actions_bit = set_actions_bit[:9*cpy_f+mass] + '0' + set_actions_bit[9*cpy_f+mass+1:]
    if cpy_f in [2, 5]: #選択してあるこまがSなら
      if board[mass+18] == '1' or board[mass+27+18] == "1": # S駒があるか
        set_actions_bit = set_actions_bit[:9*cpy_f+mass] + '0' + set_actions_bit[9*cpy_f+mass+1:]

  # actions_bitをactionsに変換
  set_actions = []
  for i, bit in enumerate(set_actions_bit):
    if bit == '1':
      set_action = int('0b_1_000000_000000000', 2)
      set_action = int(f'{set_action:016b}'[:1+(i//9)] + '1' + f'{set_action:016b}'[1+(i//9)+1:], 2) # 駒の種類の反映
      set_action = int(f'{set_action:016b}'[:1+6+(i%9)] + '1' + f'{set_action:016b}'[1+6+(i%9)+1:], 2) # 駒の位置の反映
      set_actions.append(set_action)

  return set_actions

In [5]:
# 確認

# choiseしている駒と同じ駒は行動できる。

# 先手, 手駒Lを選択時, 置ける駒はL, その位置は0~8のハズ
state = 0b_1_1_0_100000_000000000_000000000_000000000_000000000_000000000_000000000_000000000_011111111111
print("先手, 手駒Lを選択時, 置ける駒はL, その位置は0~8のハズ")
for action in leagal_actions_from_choise_state(state):
  print(f'{action:016b}')
# 先手, 手駒Mを選択時, 置ける駒はM, その位置は0~8のハズ
state = 0b_1_1_0_010000_000000000_000000000_000000000_000000000_000000000_000000000_000000000_110111111111
print("先手, 手駒Mを選択時, 置ける駒はM, その位置は0~8のハズ")
for action in leagal_actions_from_choise_state(state):
  print(f'{action:016b}')
# 後手, 手駒Sを選択時, 置ける駒はS, その位置は0~8のハズ
state = 0b_0_1_0_000001_000000000_000000000_000000000_000000000_000000000_000000000_000000000_111111111101
print("後手, 手駒Sを選択時, 置ける駒はS, その位置は0~8のハズ")
for action in leagal_actions_from_choise_state(state):
  print(f'{action:016b}')
print()

# choiseしている駒がboardの駒なら、そのマスに行動出来ない

# 先手, マス0の自駒Lを選択時, 行動位置は1~8のハズ(盤面に他の駒なし)
state = 0b_1_1_1_100000_100000000_000000000_000000000_000000000_000000000_000000000_000000000_011111111111
print("先手, マス0の自駒Lを選択時, 置ける駒はL, その位置は0~8のハズ(盤面に他の駒なし)")
for action in leagal_actions_from_choise_state(state):
  print(f'{action:016b}')
# 先手, マス3の自駒Sを選択時, 行動位置は0~2, 4~8のハズ(盤面に他の駒なし)
state = 0b_1_1_1_001000_000100000_000000000_000000000_000000000_000000000_000000000_000000000_111101111111
print("先手, マス3の自駒Sを選択時, 置ける駒はS, その位置は0~2, 4~8のハズ(盤面に他の駒なし)")
for action in leagal_actions_from_choise_state(state):
  print(f'{action:016b}')
print()

# choiseした駒以上の大きさの駒があるマスには行動できない。

# 先手, 手駒Lを選択時, 盤面には[0, 0, 0, RL, RM, RS, BL, BM, BS], 行動位置は0~2, 4~5, 7~8のハズ
state = 0b_1_1_0_100000_100000000_000100000_000010000_000001000_000000100_000000010_000000001_000101010101
print("先手, 手駒Lを選択時, 盤面には[0, 0, 0, RL, RM, RS, BL, BM, BS], 行動位置は0~2, 4~5, 7~8のハズ")
for action in leagal_actions_from_choise_state(state):
  print(f'{action:016b}')
# 先手, 手駒Mを選択時, 盤面には[0, 0, 0, RL, RM, RS, BL, BM, BS], 行動位置は0~2, 5, 8のハズ
state = 0b_1_1_0_010000_100000000_000100000_000010000_000001000_000000100_000000010_000000001_000101010101
print("先手, 手駒Mを選択時, 盤面には[0, 0, 0, RL, RM, RS, BL, BM, BS], 行動位置は0~2, 5, 8のハズ")
for action in leagal_actions_from_choise_state(state):
  print(f'{action:016b}')
# 後手, 手駒Sを選択時, 盤面には[0, 0, 0, RL, RM, RS, BL, BM, BS], 行動位置は0~2のハズ
state = 0b_1_1_0_000001_100000000_000100000_000010000_000001000_000000100_000000010_000000001_000101010101
print("後手, 手駒Sを選択時, 盤面には[0, 0, 0, RL, RM, RS, BL, BM, BS], 行動位置は0~2のハズ")
for action in leagal_actions_from_choise_state(state):
  print(f'{action:016b}')

先手, 手駒Lを選択時, 置ける駒はL, その位置は0~8のハズ
1100000100000000
1100000010000000
1100000001000000
1100000000100000
1100000000010000
1100000000001000
1100000000000100
1100000000000010
1100000000000001
先手, 手駒Mを選択時, 置ける駒はM, その位置は0~8のハズ
1010000100000000
1010000010000000
1010000001000000
1010000000100000
1010000000010000
1010000000001000
1010000000000100
1010000000000010
1010000000000001
後手, 手駒Sを選択時, 置ける駒はS, その位置は0~8のハズ
1000001100000000
1000001010000000
1000001001000000
1000001000100000
1000001000010000
1000001000001000
1000001000000100
1000001000000010
1000001000000001

先手, マス0の自駒Lを選択時, 置ける駒はL, その位置は0~8のハズ(盤面に他の駒なし)
1100000010000000
1100000001000000
1100000000100000
1100000000010000
1100000000001000
1100000000000100
1100000000000010
1100000000000001
先手, マス3の自駒Sを選択時, 置ける駒はS, その位置は0~2, 4~8のハズ(盤面に他の駒なし)
1001000100000000
1001000010000000
1001000001000000
1001000000010000
1001000000001000
1001000000000100
1001000000000010
1001000000000001

先手, 手駒Lを選択時, 盤面には[0, 0, 0, RL, RM, RS, BL, BM, BS], 行動位置は0~2, 4~5, 7~

### set_stateから合法手を作成

leagal_actions_from_set_state(set_state) -> choise_actions:  
choiseできる駒の一覧... choise_actions を作成する  
choiseできる駒とは....  
- 自分の駒しか行動できない

手駒なら,
- 手駒に動かしたい駒が１つもなければ、その駒は行動できない。
- 動かしたい駒以上の大きさの駒が配置されているマスには行動できない。

盤上の駒なら、
- 盤上の一番表面にある駒しか行動できない。
- 動かしたい駒以上の大きさの駒があるマスには行動できない。

In [6]:
def leagal_actions_from_set_state(set_state):
  is_first = f'{set_state:084b}'[0]
  board = f'{set_state:084b}'[18:18+54]
  hand = f'{set_state:084b}'[72:72+12]

  # 盤面の各マスに駒が存在するかどうか
  exist_place = f'{int(board[0:9], 2) | int(board[9:18], 2) | int(board[18:27], 2) | int(board[27:36], 2) | int(board[36:45], 2) | int(board[45:54], 2):09b}' 
  # 手駒は,
  choise_hand_actions_bit = '111111'
  for piece_type in range(6): # 手駒にその駒が１つもないなら行動不可
    if hand[piece_type*2:piece_type*2+2] == "00": 
      choise_hand_actions_bit = choise_hand_actions_bit[:piece_type] + '0' + choise_hand_actions_bit[piece_type+1:]
  if exist_place == "111111111": #Sは空きマスが1つ以上ないと選べない
    mask = "0b_110_110"
    choise_hand_actions_bit = f'{int(choise_hand_actions_bit, 2) & int(mask, 2) :06b}'
  if is_first == "1": # 先手なら、先手の駒しか動かせない
    choise_hand_actions_bit = choise_hand_actions_bit[:3] + '000'
  else: # 後手なら、後手の駒しか動かせない
    choise_hand_actions_bit = '000' + choise_hand_actions_bit[3:]
  # 盤上の駒は、
  exist_large_place = int(board[0:9], 2) | int(board[27:36], 2)
  exist_middle_place = int(board[9:18], 2) | int(board[36:45], 2)
  # 盤上の一番表面にある駒しか行動できない
  choise_board_actions_bit = board[0:9] \
   + f'{int(board[9:18], 2) & ~exist_large_place :09b}' \
   + f'{int(board[9:18], 2) & ~(exist_large_place | exist_middle_place):09b}' \
   + board[27:36] \
   + f'{int(board[36:45], 2) & ~exist_large_place :09b}' \
   + f'{int(board[9:18], 2) & ~(exist_large_place | exist_middle_place):09b}' 
  # Sは空きマスが1つ以上ないと選べない
  if exist_place == "111111111":
    mask = "0b_111111111_111111111_000000000_111111111_111111111_000000000"
    choise_board_actions_bit =  f'{int(choise_board_actions_bit, 2) & int(mask, 2) :054b}'
  if is_first == "1": # 先手なら、先手の駒しか動かせない
    choise_board_actions_bit = choise_board_actions_bit[:27] + '000000000000000000000000000'
  else: # 後手なら、後手の駒しか動かせない
    choise_board_actions_bit = '000000000000000000000000000' + choise_board_actions_bit[27:]

  # choise_hand_actions_bitをchoise_actionsへchoise_actionに変換しながら追加
  choise_actions = []
  for i, bit in enumerate(choise_hand_actions_bit): # choise_hand_actions_bit = '000_000
    if bit == '1':
      choise_action = int('0b_0_000000_000000000', 2)
      choise_action = int(f'{choise_action:016b}'[:1+i] + '1' + f'{choise_action:016b}'[1+i+1:], 2) # 駒の種類の反映
      choise_actions.append(choise_action)

  # choise_board_actions_bitをchoise_actionsへchoise_actionに変換しながら追加
  for i, bit in enumerate(choise_board_actions_bit): # choise_board_actions_bit = '000000000_000000000_000000000_000000000_000000000_000000000'
    if bit == '1':
      choise_action = int('0b_1_000000_000000000', 2)
      choise_action = int(f'{choise_action:016b}'[:1+(i//9)] + '1' + f'{choise_action:016b}'[1+(i//9)+1:], 2) # 駒の種類の反映
      choise_action = int(f'{choise_action:016b}'[:1+6+(i%9)] + '1' + f'{choise_action:016b}'[1+6+(i%9)+1:], 2) # 駒の位置の反映
      choise_actions.append(choise_action)

  return choise_actions

In [7]:
# 確認

# 手駒
# 先手, 手駒にその駒が１つもないなら行動不可
state = 0b_1_0_0_000000_000000000_000000000_000000000_000000000_000000000_000000000_000000000_110100110100
print("# 先手, 手駒にその駒が１つもないなら行動不可。(行動可能は先手の駒数が2,1つのL駒とM駒のハズ)")
for action in leagal_actions_from_set_state(state):
  print(f'{action:016b}')
print()
#Sは空きマスが1つ以上ないと選べない
state = 0b_1_0_0_000000_000000000_110000000_001100000_000000000_000011000_000000110_000000000_000011000011
print("# 先手, Sは空きマスが1つ以上ないと行動不可。(行動可能は8が空きマスなので自駒S + 盤上の自駒L,M)")
for action in leagal_actions_from_set_state(state):
  print(f'{action:016b}')
state = 0b_1_0_0_000000_000000000_110000000_001100000_000000000_000011000_000000110_000000001_000011000001
print("# 先手, Sは空きマスが1つ以上ないと行動不可。(行動可能は空きマスがないなので盤上の自駒L,M)")
for action in leagal_actions_from_set_state(state):
  print(f'{action:016b}')
print()

# 盤上の駒
# 先手、盤上の一番表面にある駒しか行動できない。(行動可能は0マスにあるL駒と1マスにあるM駒)
state = 0b_1_0_0_000000_000000000_100000000_110000000_110000000_000000000_000000000_000000000_000000111111
print("# 先手、盤上の一番表面にある駒しか行動できない。(行動可能は0マスにあるL駒と1マスにあるM駒)")
for action in leagal_actions_from_set_state(state):
  print(f'{action:016b}')

# 先手, 手駒にその駒が１つもないなら行動不可。(行動可能は先手の駒数が2,1つのL駒とM駒のハズ)
0100000000000000
0010000000000000

# 先手, Sは空きマスが1つ以上ないと行動不可。(行動可能は8が空きマスなので自駒S + 盤上の自駒L,M)
0001000000000000
1100000100000000
1100000010000000
1010000001000000
1010000000100000
# 先手, Sは空きマスが1つ以上ないと行動不可。(行動可能は空きマスがないなので盤上の自駒L,M)
1100000100000000
1100000010000000
1010000001000000
1010000000100000

# 先手、盤上の一番表面にある駒しか行動できない。(行動可能は0マスにあるL駒と1マスにあるM駒)
1100000100000000
1010000010000000


### 合法手をすべて列挙する

In [81]:
def legal_actions(state):
  actions = []
  if f'{state:084b}'[1] == '1':
    actions = leagal_actions_from_choise_state(state)
  else:
    actions = leagal_actions_from_set_state(state)
  return actions

In [82]:
# 確認

# 先手, 手駒Lを選択時, 置ける駒はL, その位置は0~8のハズ
state = 0b_1_1_0_100000_000000000_000000000_000000000_000000000_000000000_000000000_000000000_011111111111
print("先手, 手駒Lを選択時, 置ける駒はL, その位置は0~8のハズ")
for action in legal_actions(state):
  print(f'{action:016b}')

先手, 手駒Lを選択時, 置ける駒はL, その位置は0~8のハズ
1100000100000000
1100000010000000
1100000001000000
1100000000100000
1100000000010000
1100000000001000
1100000000000100
1100000000000010
1100000000000001


## state, actionから次のstateを生成する関数の作成


stateにactionを反映して、stateを作成する
```
def create_next_state(state, action) -> next_state:
	next_state = “”
	もし stateがchoise_stateなら、
		next_state = next_state_from_choise_state(choise_state, set_action)
	そうでなく、set_stateなら
		next_state = next_state_from_set_state(set_state, choise_action)
	return next_state
```
stateごとに関数を作成。  
- choise_stateから次のstateを作成 : next_state_from_choise_state(choise_state, set_action)
- set_stateから次のstateを作成 : next_state_from_set_state(set_state, choise_action)


### choise_stateから次のstateを作成

In [83]:
def next_state_from_choise_state(choise_state, set_action):
  is_first = f'{choise_state:084b}'[0]
  is_choise_board = f'{choise_state:084b}'[2]
  choise_piece_type = f'{choise_state:084b}'[3:3+6]
  choise_piece = f'{choise_state:084b}'[9:9+9]
  board = f'{choise_state:084b}'[18:18+54]
  hand = f'{choise_state:084b}'[72:72+12]

	# set_actionで選択している駒と位置を、boardで1にする。
  piece_type = f'{set_action:016b}'[1:7].find('1')
  piece_place = f'{set_action:016b}'[7:].find('1')
  board = board[:piece_type*9+piece_place] + '1' +  board[piece_type*9+piece_place+1:] 

	# 手番プレイヤーの交代 … is_firstを反転
  is_first = "0" if is_first == "1" else "1"
	# stateのモードを切り替える...is_choiseを0にする
  is_choise = "0"
  # is_choise_board, choise_piece_type, choise_piece をすべて0埋め
  is_choise_board = "0"
  choise_piece_type = "000000"
  choise_piece = "000000000"

  # next_stateの作成
  next_state = int(is_first + is_choise + is_choise_board + choise_piece_type + choise_piece + board + hand, 2)
  return next_state

In [84]:
# 確認

# (state = 先手初期浮き手駒Lのchoise盤面, action = Lを0マスに置く) => (state = 後手、先手L駒が0に配置された盤面のset状態)
state = 0b_1_1_0_100000_000000000_000000000000000000000000000000000000000000000000000000_011111111111
action = 0b_1_100000_100000000
next_state = next_state_from_choise_state(state, action)
print("正解 : next_state = 000000000000000000100000000000000000000000000000000000000000000000000000011111111111")
print( f'結果 : next_state = {next_state:084b}')
print()

# (state = 後手先手Lが0かつ浮き手駒Lのchoise盤面, action = Lを1マスに置く) => (state = 先手、先手L駒が0, 後手L駒が1に配置された盤面のset状態)
state = 0b_0_1_0_000100_000000000_100000000000000000000000000000000000000000000000000000_011111011111
action = 0b_1_000100_010000000
next_state = next_state_from_choise_state(state, action)
print("正解 : next_state = 100000000000000000100000000000000000000000000010000000000000000000000000011111011111")
print( f'結果 : next_state = {next_state:084b}')
print()

# (state = 先手先手L0後手L1浮き盤駒Lのchoise盤面, action = Lを2マスに置く) => (state = 後手、先手L駒が2、後手L駒が1に配置された盤面のset状態)
state = 0b_1_1_1_100000_100000000_000000000000000000000000000010000000000000000000000000_011111011111
action = 0b_1_100000_001000000
next_state = next_state_from_choise_state(state, action)
print("正解 : next_state = 000000000000000000001000000000000000000000000010000000000000000000000000011111011111")
print( f'結果 : next_state = {next_state:084b}')
print()

正解 : next_state = 000000000000000000100000000000000000000000000000000000000000000000000000011111111111
結果 : next_state = 000000000000000000100000000000000000000000000000000000000000000000000000011111111111

正解 : next_state = 100000000000000000100000000000000000000000000010000000000000000000000000011111011111
結果 : next_state = 100000000000000000100000000000000000000000000010000000000000000000000000011111011111

正解 : next_state = 000000000000000000001000000000000000000000000010000000000000000000000000011111011111
結果 : next_state = 000000000000000000001000000000000000000000000010000000000000000000000000011111011111



### set_stateから次のstateを作成

In [85]:
def next_state_from_set_state(set_state, choise_action):
  is_first = f'{set_state:084b}'[0]
  is_choise_board = f'{set_state:084b}'[2]
  choise_piece_type = f'{set_state:084b}'[3:3+6]
  choise_piece = f'{set_state:084b}'[9:9+9]
  board = f'{set_state:084b}'[18:18+54]
  hand = f'{set_state:084b}'[72:72+12]

	# stateのモードを切り替える...is_choiseを1にする
  is_choise = "1"
  # choise_actionをis_choise_board, choise_piece_type, choise_piece に反映する
  is_choise_board = f'{choise_action:016b}'[0]
  choise_piece_type = f'{choise_action:016b}'[1:7]
  choise_piece =  f'{choise_action:016b}'[7:16]
  # 選択した駒をboard, handから取り除く
  piece_type = f'{choise_action:016b}'[1:7].find('1')
  piece_place = f'{choise_action:016b}'[7:].find('1')
  if f'{choise_action:016b}'[0] == "1":   # choise_actionで盤の駒を選択しているなら
    # choise_actionで選択している駒と位置を、boardで0にする。
    board = board[:piece_type*9+piece_place] + '0' +  board[piece_type*9+piece_place+1:] 
  # choise_actionで選択している駒の数を、handで1減らす。
  # “00”...その駒は持ってない, “01”...一つ持ってる, “11”...２つ持ってる	
  if hand[piece_type*2:piece_type*2+2] == "11":
    hand = hand[:piece_type*2] + "01" + hand[piece_type*2+2:]
  else: # hand[piece_type*2:piece_type*2+2] == "01":
    hand = hand[:piece_type*2] + "00" + hand[piece_type*2+2:]

  # next_stateの作成
  next_state = int(is_first + is_choise + is_choise_board + choise_piece_type + choise_piece + board + hand, 2)
  return next_state

### 正規化

In [86]:
def convert_normalization_state(state):
  is_first = f'{state:084b}'[0]
  is_choise = f'{state:084b}'[1]
  is_choise_board = f'{state:084b}'[2]
  choise_piece_type = f'{state:084b}'[3:3+6]
  choise_piece = f'{state:084b}'[9:9+9]
  board = f'{state:084b}'[18:18+54]
  hand = f'{state:084b}'[72:72+12]

  # 変換表を定義
  convert_tables = [[6,3,0,7,4,1,8,5,2],[8,7,6,5,4,3,2,1,0],
                    [2,5,8,1,4,7,0,3,6],[2,1,0,5,4,3,8,7,6],
                    [6,7,8,3,4,5,0,1,2],[0,3,6,1,4,7,2,5,8],[8,5,2,7,4,1,6,3,0]]

  # 変換表でboardを変換したcand_boardを作成し、boardと比較して小さいなら上書きする
  # もしより小さい候補が見つかったら、それと同じ変換表でchoise_pieceも変換する
  board_list = list(board)
  choise_piece_list = list(choise_piece)
  for convert_table in convert_tables:
    cand_board_list = [0] * 54
    for i in range(6):
      for after_num, before_num in enumerate(convert_table):
        cand_board_list[i*9 + after_num] = board_list[i*9 + before_num]
    # print("cand_board : " + "".join(cand_board_list))  # デバッグ用
    # board_list と cand_board_list を比較
    if board_list > cand_board_list:
      board_list = cand_board_list # 上書き
      for after_num, before_num in enumerate(convert_table): # choise_pieceの変換
        choise_piece_list[after_num] = choise_piece_list[before_num]

  # board_list, choise_piece_listをboard, choise_pieceに変換
  board = "".join(board_list)
  choise_piece = "".join(choise_piece_list)

  normalization_state = int(is_first + is_choise + is_choise_board + choise_piece_type + choise_piece + board + hand, 2)
  return normalization_state

In [87]:
# 確認
# 先手, 手駒Lを選択時, 置ける駒はL, その位置は0~8のハズ
state = 0b_1_0_0_000000_000000000_100000000_000000000_000000000_000000000_000000000_000000000_011111111111
print("先手L駒0マスにおく -> 先手L駒は8マスに")
print(f'結果 : normalization_state = {convert_normalization_state(state):084b}')

先手L駒0マスにおく -> 先手L駒は8マスに
結果 : normalization_state = 100000000000000000000000001000000000000000000000000000000000000000000000011111111111


### stateのモードに関わらず、次のstateを生成する

In [88]:
def create_next_state(state, action):
  is_choise = f'{state:084b}'[1]
  next_state = 0
  if is_choise == "1":
    next_state = next_state_from_choise_state(state, action)
  else:
    next_state = next_state_from_set_state(state, action)
  next_state = convert_normalization_state(next_state) # 下記確認ではコメントアウト
  return next_state

In [89]:
# 確認
# ※ 正規化前の確認。正規化したら正解と結果は異なる。
# (state = 先手初期set盤面, action = 手駒Lを選択) => (state = 先手、初期choise盤面、手駒L選択の状態)
state = 0b_1_0_0_000000_000000000_000000000000000000000000000000000000000000000000000000_111111111111
action = 0b_0_100000_000000000
next_state = create_next_state(state, action)
print("正解 : next_state = 110100000000000000000000000000000000000000000000000000000000000000000000011111111111")
print( f'結果 : next_state = {next_state:084b}')
print()

# (state = 先手初期浮き手駒Lのchoise盤面, action = Lを0マスに置く) => (state = 後手、先手L駒が0に配置された盤面のset状態)
state = 0b_1_1_0_100000_000000000_000000000000000000000000000000000000000000000000000000_011111111111
action = 0b_1_100000_100000000
next_state = create_next_state(state, action)
print("正解 : next_state = 000000000000000000100000000000000000000000000000000000000000000000000000011111111111")
print( f'結果 : next_state = {next_state:084b}')
print()

# (state = 後手0マスに先手L駒のset盤面, action = 手駒Mを選択) => (state = 後手、マスに先手L駒の盤面、手駒M選択のchoise状態)
state = 0b_0_0_0_000000_000000000_100000000000000000000000000000000000000000000000000000_011111111111
action = 0b_0_000010_000000000
next_state = create_next_state(state, action)
print("正解 : next_state = 010000010000000000100000000000000000000000000000000000000000000000000000011111110111")
print( f'結果 : next_state = {next_state:084b}')
print()

# (state = 後手先手Lが0かつ浮き手駒Lのchoise盤面, action = Lを1マスに置く) => (state = 先手、先手L駒が0, 後手L駒が1に配置された盤面のset状態)
state = 0b_0_1_0_000100_000000000_100000000000000000000000000000000000000000000000000000_011111011111
action = 0b_1_000100_010000000
next_state = create_next_state(state, action)
print("正解 : next_state = 100000000000000000100000000000000000000000000010000000000000000000000000011111011111")
print( f'結果 : next_state = {next_state:084b}')
print()

# (state = 先手先手L0後手L1のset盤面, action = 盤駒Lを選択) => (state = 先手、後手L1浮き盤駒Lのchoise盤面、手駒L選択の状態)
state = 0b_1_0_0_000000_000000000_100000000000000000000000000010000000000000000000000000_011111011111
action = 0b_1_100000_100000000
next_state = create_next_state(state, action)
print("正解 : next_state = 111100000100000000000000000000000000000000000010000000000000000000000000011111011111")
print( f'結果 : next_state = {next_state:084b}')
print()

# (state = 先手先手L0後手L1浮き盤駒Lのchoise盤面, action = Lを2マスに置く) => (state = 後手、先手L駒が2、後手L駒が1に配置された盤面のset状態)
state = 0b_1_1_1_100000_100000000_000000000000000000000000000010000000000000000000000000_011111011111
action = 0b_1_100000_001000000
next_state = create_next_state(state, action)
print("正解 : next_state = 000000000000000000001000000000000000000000000010000000000000000000000000011111011111")
print( f'結果 : next_state = {next_state:084b}')
print()

正解 : next_state = 110100000000000000000000000000000000000000000000000000000000000000000000011111111111
結果 : next_state = 110100000000000000000000000000000000000000000000000000000000000000000000011111111111

正解 : next_state = 000000000000000000100000000000000000000000000000000000000000000000000000011111111111
結果 : next_state = 000000000000000000000000001000000000000000000000000000000000000000000000011111111111

正解 : next_state = 010000010000000000100000000000000000000000000000000000000000000000000000011111110111
結果 : next_state = 010000010000000000000000001000000000000000000000000000000000000000000000011111110111

正解 : next_state = 100000000000000000100000000000000000000000000010000000000000000000000000011111011111
結果 : next_state = 100000000000000000000000001000000000000000000000000010000000000000000000011111011111

正解 : next_state = 111100000100000000000000000000000000000000000010000000000000000000000000011111011111
結果 : next_state = 111100000000000000000000000000000000000000000000000

## 結合テスト



actions が空になるまで、初期盤面からnext_stateを探索していく。  
ここでは、`action = actions[0]`とする。

In [98]:
# 確認
state = 0b_1_0_0_000000_000000000_000000000000000000000000000000000000000000000000000000_111111111111
print("0番目")
print("状態 : {}".format(state))
action = legal_actions(state)[0]
print("行動 : {}".format(action))
for i in range(1,10):
  print(str(i) + "番目")
  state = create_next_state(state, action)
  print("状態 : {}".format(state))
  action = legal_actions(state)[0]
  print("行動 : {}".format(action))
  print()


0番目
状態 : 9671406556917033397653503
行動 : 16384
1番目
状態 : 15716035654990179271182335
行動 : 49408

2番目
状態 : 144115188075857919
行動 : 2048

3番目
状態 : 4986819150025533421520863
行動 : 35072

4番目
状態 : 9671406701032496351414239
行動 : 16384

5番目
状態 : 15716035799105642224944095
行動 : 49280

6番目
状態 : 1729382325629748191
行動 : 2048

7番目
状態 : 4986820735292670975411151
行動 : 35072

8番目
状態 : 9671408286299633905304527
行動 : 8192

9番目
状態 : 15111574474565465191481807
行動 : 41088



## 盤面の判定をする関数の生成 

盤面に対して、lineがあるかどうか、あるならどちらの駒かを確認する。
```
def check_line(state) -> [is_line, is_first_line]:
	stateのboard部分からsurface_boardを作成
	# プレイヤーごとに３目並べの判定をし、もし揃っていたら
		is_line = True
		is_first_line = そのlineがfirst_playerのものかどうか
	# そろってなかったら, is_lineはFalse
	return [is_line, is_first_line]
```

surface_board
 … boardで上に駒が重なっている駒を0にした。

追記: check_result(state) -> [is_done, winner]:に変更  
- is_done : lineがあるか
- winner : 先手なら0, 後手なら1

In [105]:
# 勝敗の有無
def is_win(single_surface):
  s = single_surface
  # 横, 縦, 左斜め, 右斜めのラインを調べる
  check_lines = [[0,1,2], [3,4,5], [6,7,8], [0,3,6], [1,4,7], [2,5,8], [0,4,8], [2,4,6]]
  for check_line in check_lines:
    if s[check_line[0]] and s[check_line[1]] and s[check_line[2]]:
      return True;
  return False;
# 表面の駒だけbitが立つboardを作成する
def create_surface(board):
  # boardの変換
  board_list_list = [0] * 6
  for i in range(6):
    board_list_list[i] = list(board[i*9:i*9+9])
  board_surface = [[0,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,0],
                    [0,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,0],
                    [0,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,0]]
  # S駒を反映させる
  board_surface[2] = board_list_list[2]
  board_surface[5] = board_list_list[5]
  # このとき、おいた場所により小さい駒があったら、その駒を0にする
  # １マスずつ見ていく
  for place in range(9):
    # M駒を反映させる
    if board_list_list[1][place] == "1" or board_list_list[4][place] == "1":
      if board_list_list[1][place] == "1":
        board_surface[1][place] = 1
      if board_list_list[4][place] == "1":
        board_surface[4][place] = 1
      board_surface[2][place] = 0 # S駒を0にする
      board_surface[5][place] = 0
    # L駒を反映させる
    if board_list_list[0][place] == "1" or board_list_list[3][place] == "1":
      if board_list_list[0][place] == "1":
        board_surface[0][place] = 1
      if board_list_list[3][place] == "1":
        board_surface[3][place] = 1
      board_surface[1][place] = 0 # M駒を0にする
      board_surface[4][place] = 0	
      board_surface[2][place] = 0 # S駒を0にする
      board_surface[5][place] = 0
  return board_surface

#勝敗の有無、勝者を確認する
def check_result(state):
  board = f'{state:084b}'[18:18+54]
  board_surface = create_surface(board)
  is_done = 0 # 決着がついているなら1を返す
  winner = 0 # 先手は0, 後手は1
  # 内部でboard_surface[0,1,2]とboard_surface[3,4,5]を合成
  single_surfaces = [[0,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,0]];
  for i in range(9):
    if board_surface[0][i] == 1 or board_surface[1][i] == 1 or board_surface[2][i] == 1:
      single_surfaces[0][i] = 1
    elif board_surface[3][i] == 1 or board_surface[4][i] == 1 or board_surface[5][i] == 1:
      single_surfaces[1][i] = 1
  is_done = is_win(single_surfaces[0]) == 1 or is_win(single_surfaces[1]) == 1
  winner = 1 if is_win(single_surfaces[1]) == 1 else 0
  return [is_done, winner]

# ゲーム木の作成

## 説明

ノードの３状態:
  - 未発見
    unsolved, solvedのいずれにもnext_nodeが存在しない
  - 未訪問
    unsolvedに存在する
  - 既訪問
    solvedに存在する

処理:
- 初期状態"____"を未訪問にする.
- 未訪問キューが空になるまで以下を行う # BFS開始
  1. 未訪問キューから先頭のノードをcurrent_nodeとしてpopする.
  1. 盤面の勝敗判定を行う。
    1. もし、勝敗がついている盤面なら、
      1. RESULTに勝敗を記録
    1. 勝敗がついていない盤面なら
      1. current_nodeを探索し、次のノード(next_nodes)全てをnext_nodesとして列挙する.
      1. next_nodesの要素next_nodeそれぞれに対して、以下を行う.
        1. もし、next_nodeが未発見ならば、
          1. そのノードを未訪問にする.
        1. そうではなく、発見済みならば
          1. 特になし.
        1. next_node.previous_nodeにcurrent_nodeを追加する.
        1. current_node.next_node.にnext_nodeを追加する.
  1. ノード（current_node）を既訪問にする.

1. solvedをcsvで書き出し.

## 実装

### 準備

In [107]:
# ドライブのマウント
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [108]:
# csvの読み込むためのモジュール
import pandas as pd
from pandas import DataFrame
import numpy as np
from tabulate import tabulate # pandasのdfをきれいに出力するためのモジュール

### 探索

In [120]:
### BFSでゲーム木を作成するプログラム ### 

###

# stateを入力すると、その盤面の勝敗を返す関数
def resultcheck(state: str) -> str:
  # return 一覧
    # "o_win"
    # "x_win"
    # "draw"
    # ""
  result = ""
  return result

# state を入力すると、次のstateのリストを出力する関数

def nextStates(state: str) -> list: 
  return next_states

### 設定ここから ###
printFlag = True
pd.options.display.max_rows = 100
pd.options.display.max_columns = 100
### 設定ここまで ###

###mainここから

# unsolvedDf, solvedDfの初期化
if printFlag:
  print("===")
  print("プログラム開始")
  print("===")
  print()
  print("データを初期化します")
cols = ["PREVIOUS_STATES", "STATE", "NEXT_STATES", "RESULT"] #[前の状態list， 状態, 次の状態list]
df = pd.DataFrame(index=[], columns=cols)
df.set_index("STATE")
unsolvedDf = df
solvedDf = df
if printFlag:
  print("データを初期化しました")
  print()

# 初期状態"____"をunsolvedに追加する。unsolvedに積まれているノードは未訪問.
if printFlag:
  print("===")
  print("BFSの準備")
  print("===")
  print()
  print("初期状態をセットします")
init_state = 0b_1_0_0_000000_000000000_000000000000000000000000000000000000000000000000000000_111111111111
previous_state = ""
unsolvedDf = unsolvedDf.append(pd.Series([[previous_state], init_state, "unsolved", ""], index=df.columns, name=init_state))
if printFlag:
  print("初期状態をセットしました") # 確認
  print("確認[UNSOLVED_DF]:") # 確認
  print(unsolvedDf) # 確認
  print() # 確認

# unsolvedが空になるまで以下を行う. BFS開始
if printFlag:
  print("===")
  print("BFSを開始します")
  print("===")
  print()
for _ in range(1000): # while len(unsolvedDf) > 0: # 開発のためにfor文にしている。
  pass;
  # # unsolvedDfから先頭のノードをpopする。
  # if len(unsolvedDf) <= 0:
  #   break;
  # current_node = unsolvedDf.iloc[0]  # 先頭のノード(current_node)を抽出。
  # unsolvedDf.drop(unsolvedDf.index[0], inplace=True)  # 抽出したノードをunsolvedから削除。
  # # 勝敗の確認
  # result = resultcheck(current_node.STATE)
  # # 勝敗確定盤面なら
  # if result != "":
  #   current_node.RESULT = result
  #   current_node.NEXT_STATES = []
  # else:  # 勝敗確定盤面でないなら
  #   # 先頭のノード(current_node)から次のノード(next_nodes)を探索する。
  #   next_states = nextStates(current_node.STATE) # 次のノードの探索結果
  #   current_node.NEXT_STATES = next_states # current_nodeのNEXT_STATESに探索結果を反映
  #   # 探索した全ての状態について、以下を行う。
  #   if printFlag:
  #     print("unsolvedDfからpopされたノード'{}'の探索を行います".format(current_node.STATE))
  #   if len(next_states) <= 0:
  #     if printFlag:
  #       print("    探索結果: このノードは末端です")
  #   for next_state in next_states:
  #     # もし、next_nodeが未発見ならば # unsolved, solvedのいずれにもnext_nodeが存在しない
  #     if (next_state not in unsolvedDf.STATE.values) and (next_state not in solvedDf.STATE.values):
  #       if next_state == current_node.STATE: # 次のノードが自身と同一
  #         if printFlag:
  #           print("探索結果: 自身のノード'{}'と同一です".format(next_state))
  #         continue;
  #       else:
  #         if printFlag:
  #           print("    探索結果: 未発見のノード'{}'です".format(next_state))
  #         # T)そのノードを未訪問にする。 # unsolvedに追加
  #         previous_state = [current_node.STATE]
  #         next_node = pd.Series([previous_state, next_state, "unsolved", ""], index=df.columns, name=next_state) # next_nodeの作成
  #         unsolvedDf = unsolvedDf.append(next_node)
  #     else:  # F)そうではなく、発見済みならば
  #       if printFlag:
  #         print("    探索結果: 発見済みのノード'{}'です".format(next_state))
  #       #これを既に登録されていたノードのprevious_stateに追加する。
  #       previous_state = [current_node.STATE] 
  #       if next_state in unsolvedDf.STATE.values: # unsolvedDfに存在
  #         if printFlag:
  #           print("        これはunsolvedに存在しています")
  #         # unsolvedDf[unsolvedDf.STATE.values == next_state])にprevious_stateを追加する
  #         tmp = unsolvedDf.loc[next_state, "PREVIOUS_STATES"]
  #         tmp.append(previous_state[0])       
  #         unsolvedDf.loc[next_state, "PREVIOUS_STATES"] = tmp
  #       elif next_state in solvedDf.STATE.values:# solveDfに存在
  #         if printFlag:
  #           print("        これはsolvedに存在しています")
  #         # solvedDf[solvedDf.STATE.values == next_state])にprevious_stateを追加する
  #         tmp = solvedDf.loc[next_state, "PREVIOUS_STATES"]
  #         tmp.append(previous_state[0])       
  #         solvedDf.loc[next_state, "PREVIOUS_STATES"] = tmp
  #       else: # 何らかの理由で漏れた状態
  #         print("        エラー")
  # # 現在のノード（current_node）をsolvedDfに追加する。solvedDfのノードは既訪問。 
  # solvedDf = solvedDf.append(current_node)

if printFlag:
  print()
  print("BFSが終了しました")
  print()

# 結果確認
print("===")
print("結果確認")
print("===")
print()
print("確認[unsolvedDf]:")
print()
print(tabulate(unsolvedDf, unsolvedDf.columns,tablefmt='github', showindex=True))
print()
print("確認[solvedDf]:")
print()
print(tabulate(solvedDf, solvedDf.columns,tablefmt='github', showindex=True))
print()


### mainここまで

===
プログラム開始
===

データを初期化します
データを初期化しました

===
BFSの準備
===

初期状態をセットします
初期状態をセットしました
確認[UNSOLVED_DF]:
                          PREVIOUS_STATES                      STATE  \
9671406556917033397653503              []  9671406556917033397653503   

                          NEXT_STATES RESULT  
9671406556917033397653503    unsolved         

===
BFSを開始します
===


BFSが終了しました

===
結果確認
===

確認[unsolvedDf]:

|                           | PREVIOUS_STATES   |                     STATE | NEXT_STATES   | RESULT   |
|---------------------------|-------------------|---------------------------|---------------|----------|
| 9671406556917033397653503 | ['']              | 9671406556917033397653503 | unsolved      |          |

確認[solvedDf]:

| PREVIOUS_STATES   | STATE   | NEXT_STATES   | RESULT   |
|-------------------|---------|---------------|----------|

